In [1]:
import pandas as pd
import json 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# definition of the extraction data
# to extract car overview
def car_overview_dict_creator(overview):
    overview_dict = json.loads(overview.replace("'", '"').replace('None', "1"))
    all_overviews = {}
    for dictionary in overview_dict['top']:
        all_overviews.update({dictionary['key']: dictionary['value']})
    return all_overviews
# to extract car feature
def car_feature_dict_creator(feature):
  all_features = {}
  feature_dict = json.loads(feature.replace("'", '"').replace('None', "1"))
  for dictionary in feature_dict['top']:
    all_features.update({dictionary['value']: True})
    for section in feature_dict.get('data', []):
      for item in section.get('list', []):
        all_features.update({item['value']: True})
  return all_features
# extract car specs
def car_spec_dict_creator(spec):
    all_specs = {}
    specs_dict = json.loads(spec.replace("'", '"').replace('None', "null"))
    for dictionary in specs_dict.get('top', []):
      all_specs.update({dictionary['key']: dictionary['value']})
    # Extract nested specifications
    for dictionary in specs_dict.get('data', []):
      for item in dictionary.get('list', []):
        all_specs.update({item['key'] : item['value']})
    return all_specs
df = pd.read_excel("C:/Users/anant/OneDrive/Desktop/jupyter/PROJECT 3/delhi_cars.xlsx")
#to extract car detail
df.new_car_detail=df['new_car_detail'].apply(lambda x : json.loads(x.replace("'",'"').replace('None',"1")))
df_car_detail = pd.DataFrame(df.new_car_detail.to_list())
# to extract car overview
df.new_car_overview = df['new_car_overview'].apply(lambda overview: car_overview_dict_creator(overview))
df_car_overview = pd.DataFrame(df.new_car_overview.to_list())
# to extract car feature
feature = pd.DataFrame(df.new_car_feature)
feature = feature.rename(columns={feature.columns[0]: 'feature'})
df = df.merge(feature, left_index=True, right_index=True)
df['feature_dict'] = df['feature'].apply(car_feature_dict_creator)
feature_df = pd.json_normalize(df['feature_dict']).fillna(False).astype(int)
# extract car specs
df['new_car_specs'] = df['new_car_specs'].apply(lambda spec: car_spec_dict_creator(spec))
df_car_specs = pd.DataFrame(df['new_car_specs'].tolist())
df_bangalore = pd.concat([df_car_detail,df_car_overview,feature_df, df_car_specs], axis=1)
df_bangalore ['city'] = 'Delhi'
df_bangalore.to_csv('structured_delhi_cars.csv', index=False)

C:\Users\anant\AppData\Local\Temp\ipykernel_36272\3301141154.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  feature_df = pd.json_normalize(df['feature_dict']).fillna(False).astype(int)
